# Modelo LightGBM

## Importar datos limpios

In [4]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Cargar el archivo CSV 
client_df = pd.read_csv('../data/raw/base_clientes_final.csv', parse_dates=['fecha_nacimiento', 'fecha_alta'])
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     1000 non-null   object        
 1   fecha_nacimiento       1000 non-null   datetime64[ns]
 2   fecha_alta             1000 non-null   datetime64[ns]
 3   id_municipio           1000 non-null   int64         
 4   id_estado              1000 non-null   int64         
 5   tipo_persona           1000 non-null   object        
 6   genero                 1000 non-null   object        
 7   actividad_empresarial  1000 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 62.6+ KB


In [5]:
# Cargar el archivo CSV usando la ruta absoluta
tx_df = pd.read_csv('../data/raw/base_transacciones_final.csv', parse_dates=['fecha'])
tx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346011 entries, 0 to 346010
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             346011 non-null  object        
 1   fecha          346011 non-null  datetime64[ns]
 2   comercio       346011 non-null  object        
 3   giro_comercio  340423 non-null  object        
 4   tipo_venta     346011 non-null  object        
 5   monto          346011 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 15.8+ MB


# Análisis

In [6]:
# --- Step 1: Aggregate monthly expense per client ---
tx_df['month'] = tx_df['fecha'].dt.to_period('M')
monthly_expenses = tx_df.groupby(['id', 'month'])['monto'].sum().reset_index()
monthly_expenses.rename(columns={'monto': 'monto_total'}, inplace=True)
monthly_expenses['month'] = monthly_expenses['month'].dt.to_timestamp()

In [ ]:
# --- Step 2: Add features ---
df = monthly_expenses.merge(client_df, on='id', how='left')
df

In [9]:
# Temporal features
df['month_num'] = df['month'].dt.month
df['year'] = df['month'].dt.year

# Sort for rolling calculations
df = df.sort_values(['id', 'month'])

In [10]:
# Lag & rolling features
df['monto_prev'] = df.groupby('id')['monto_total'].shift(1)
df['rolling_3'] = df.groupby('id')['monto_total'].rolling(3).mean().reset_index(0, drop=True)

# Drop NaNs from rolling/lags
df = df.dropna(subset=['monto_prev', 'rolling_3'])

## Configuración del test

In [11]:
# --- Step 3: Train/Test split (last month is test) ---
last_month = df['month'].max()
test_month = last_month
train = df[df['month'] < test_month]
test = df[df['month'] == test_month]

### Inputs del modelo

In [12]:
# --- Step 4: Prepare model inputs ---
categorical = ['id', 'tipo_persona', 'genero', 'actividad_empresarial']
for col in categorical:
    train[col] = train[col].astype('category')
    test[col] = test[col].astype('category')

features = ['month_num', 'year', 'monto_prev', 'rolling_3', 'id', 
            'id_municipio', 'id_estado', 'tipo_persona', 'genero', 'actividad_empresarial']
target = 'monto_total'

C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_18992\3457489763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = train[col].astype('category')
C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_18992\3457489763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col] = test[col].astype('category')
C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_18992\3457489763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

## Entrenamiento del modelo

In [13]:
# --- Step 5: Train the model ---
model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
model.fit(train[features], train[target])

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000949 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1737
[LightGBM] [Info] Number of data points in the train set: 9975, number of used features: 9
[LightGBM] [Info] Start training from score 1156.037994


LGBMRegressor(random_state=42)

### Pickle

In [ ]:
import pickle

# Save the model to a file
# with open("lightgbm_model.pkl", "wb") as f:
#     pickle.dump(model, f)

print("Model saved as 'lightgbm_model.pkl'")


Model saved as 'lightgbm_model.pkl'


## Resultados

In [14]:
# --- Step 6: Predict & Evaluate ---
test['prediction'] = model.predict(test[features])
rmse = np.sqrt(mean_squared_error(test[target], test['prediction']))
print(f"RMSE on test month ({test_month.date()}): {rmse:.2f}")

RMSE on test month (2023-01-01): 957.03


C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_18992\542704420.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['prediction'] = model.predict(test[features])


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# --- Global metrics ---
y_true = test[target]
y_pred = test['prediction']

mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
r2 = r2_score(y_true, y_pred)

print(f"Global Evaluation Metrics for test month {test_month.date()}:")
print(f"  RMSE : {rmse:.2f}")
print(f"  MAE  : {mae:.2f}")
print(f"  MAPE : {mape:.2f}%")
print(f"  R²   : {r2:.4f}")

Global Evaluation Metrics for test month 2023-01-01:
  RMSE : 957.03
  MAE  : 400.25
  MAPE : 43.18%
  R²   : 0.6604


In [19]:
# --- Per-client actual vs predicted spending ---
client_summary = test.groupby('id').agg(
    actual_avg=('monto_total', 'mean'),
    predicted_avg=('prediction', 'mean')
).reset_index()

# Calculate individual errors
client_summary['error'] = client_summary['predicted_avg'] - client_summary['actual_avg']
client_summary['abs_error'] = client_summary['error'].abs()
client_summary['perc_error'] = (client_summary['abs_error'] / client_summary['actual_avg']) * 100

# Show summary stats
print("\nClient-level Error Summary:")
print(client_summary[['abs_error', 'perc_error']].describe())

# Save for inspection
client_summary.to_csv("client_avg_comparison.csv", index=False)
print("Saved per-client average comparison to 'client_avg_comparison.csv'")



Client-level Error Summary:
          abs_error   perc_error
count   1000.000000  1000.000000
mean     400.251412    43.178731
std      869.752002    83.494760
min        0.112368     0.028103
25%       90.019618    10.792109
50%      199.824543    22.956660
75%      399.858495    42.094539
max    17300.226134  1247.051451
Saved per-client average comparison to 'client_avg_comparison.csv'


C:\Users\Juan Marco\AppData\Local\Temp\ipykernel_18992\1910192165.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  client_summary = test.groupby('id').agg(


In [16]:
# Obtener el promedio de los gastos mensuales por cliente
monthly_avg = df.groupby('id')['monto_total'].mean().reset_index()
monthly_avg.rename(columns={'monto_total': 'avg_monto_total'}, inplace=True)

In [17]:
monthly_avg

,id,avg_monto_total
0,003d9abe467a91847d566cf455bd2d7d6c8f7e75,1053.646364
1,0066f66c7fb0d1102f9f924a56ff65a5fcaf3cea,2042.773636
2,0132462ee1acf1c37e7411b20bac94e1d19959df,888.748182
3,0172891ba3000c3150a6985265517678ebbf3b39,828.090909
4,01d12d1c5b58348995b71cf1d5eb9257fa64d95b,1542.042727
...,...,...
995,fdf02d0a51b8680e60bdb3aec28553729f2d4145,590.640909
996,fe25435085da4e8b314592f5aab3655699ea1894,1061.270909
997,fe29a4b444e0c24c7efb5c4c5209e25e08893716,615.029091
998,fe9415c62193f2d430a0340c31064ec512b27c8c,1169.754545


In [15]:
# --- Step 7: Save results ---
test[['id', 'month', 'monto_total', 'prediction']].to_csv("client_predictions.csv", index=False)
print("Predictions saved to 'client_predictions.csv'")


Predictions saved to 'client_predictions.csv'
